In [1]:
import pickle
import numpy as np
import neat
import os
import keyboard

#from game_config import game_config
# MLAGENTS stuff
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.base_env import ActionTuple
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel

In [2]:
engine_config_channel = EngineConfigurationChannel()
from comunication_channel import AgentLogChannel
agent_count_channel = AgentLogChannel()
engine_config_channel.set_configuration_parameters(time_scale=1)

env_path = "../Builds/train-env/autonomous-drone.exe"

env = UnityEnvironment(file_name=None, seed=0, no_graphics=False, side_channels=[engine_config_channel, agent_count_channel])
env.reset()

In [3]:
behavior_specs = env.behavior_specs
print(f"Behaviour specs {behavior_specs}")
behavior_name = list(behavior_specs)[0]
spec = env.behavior_specs[behavior_name]

print(f"Name of the behavior : {behavior_name}")
print("Number of observations : ", len(spec.observation_specs)) # vector if 1

Behaviour specs <mlagents_envs.base_env.BehaviorMapping object at 0x00000270F49C6D70>
Name of the behavior : WindControl?team=0
Number of observations :  1


In [4]:
# Is the Action continuous or multi-discrete ?
if spec.action_spec.continuous_size > 0:
  print(f"There are {spec.action_spec.continuous_size} continuous actions")
if spec.action_spec.is_discrete():
  print(f"There are {spec.action_spec.discrete_size} discrete actions")

There are 4 continuous actions


In [5]:
def map_agent_ids(decision_steps):
    """
    Map agent ids between NEAT and UNITY.

    Args:
        decision_steps: An iterable containing decision steps.

    Returns:
        A tuple of two dictionaries: (unity_to_neat_map, neat_to_unity_map)
    """
    unity_to_neat_map = {}
    neat_to_unity_map = {}
    id_count = 0
    for step in decision_steps:
        unity_to_neat_map[step] = id_count
        neat_to_unity_map[id_count] = step
        id_count += 1
    return unity_to_neat_map, neat_to_unity_map

In [6]:
def set_agents_and_double_reset(num_agents: int):
    agent_count_channel.send_int(data=num_agents) 
    env.reset()
    env.reset()

In [7]:
set_agents_and_double_reset(2)

In [8]:
import time
import random
import math

def eval_agent(genome, cfg):
    i = 0
    send_array = [2, 1, 2]

    while True:
        #policy = neat.ctrnn.CTRNN.create(genome, cfg, 0.02)
        policy = neat.nn.RecurrentNetwork.create(genome, cfg)
        set_agents_and_double_reset(random.randint(1, 1))
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        agent_count = len(decision_steps.agent_id)
        policies = [policy] * agent_count

        unity_to_neat_map, neat_to_unity_map = map_agent_ids(decision_steps)

        done = False  # Vectorized initialization
        removed_agents = []

        episode_rewards = [0] * agent_count
        print(f"Agent count: {agent_count}")
        while not done:
            # if keyboard.is_pressed('q'):  # Check if 'q' is pressed
            #     env.close()
            #     return
            for agent in decision_steps:
                if unity_to_neat_map[agent] not in removed_agents:
                    nn_input = np.asarray(decision_steps[agent].obs[:])
                    #print(f"{agent} {nn_input}")
                    actions = policies[unity_to_neat_map[agent]].activate(nn_input[0])
                    #print(f"{agent} {actions}")
                    #actions = [0, 0.5,0.5, 1]
                    continous_actions = np.asarray([actions])
                    continous_actions = np.clip(continous_actions, -1, 1)

                    action_tuple = ActionTuple(discrete=None, continuous=continous_actions)
                    env.set_action_for_agent(behavior_name=behavior_name, 
                                            agent_id=agent, 
                                            action=action_tuple)
            env.step()
            decision_steps, terminal_steps = env.get_steps(behavior_name)
            for agent in range(agent_count):
                if agent not in removed_agents:
                    local_agent = neat_to_unity_map[agent]
                    if local_agent in terminal_steps:
                        episode_rewards[agent] += terminal_steps[local_agent].reward
                        removed_agents.append(agent)
                    elif local_agent in decision_steps:
                        episode_rewards[agent] += decision_steps[local_agent].reward                             
            if len(removed_agents) >= agent_count:
                print(".") 
                done = True
        for reward in episode_rewards:
            print(reward)
        #while keyboard.is_pressed('c') == False:
        #    time.sleep(0.1)

In [9]:
save_nn_destination = '../result/best.pkl'

with open(save_nn_destination, "rb") as f:
    genome = pickle.load(f)
    genome = genome[1]
    print(genome)
print(genome.fitness)
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                    neat.DefaultSpeciesSet, neat.DefaultStagnation,
                    '../configs/config-recurrent-wind')
# Save best genome
# with open(f'best.pkl', 'wb') as f:
#    pickle.dump(genome, f)
eval_agent(genome, config)

Key: 9057
Fitness: 51.17962782316354
Nodes:
	0 DefaultNodeGene(key=0, bias=0.015570083272938745, response=1.0, activation=clamped, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=0.23292698012109525, response=1.0, activation=clamped, aggregation=sum)
	2 DefaultNodeGene(key=2, bias=0.035022534895158206, response=1.0, activation=clamped, aggregation=sum)
	3 DefaultNodeGene(key=3, bias=0.1409022761420273, response=1.0, activation=clamped, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-9, 0), weight=-0.7703546474872193, enabled=False)
	DefaultConnectionGene(key=(-8, 1), weight=-0.20100054938288375, enabled=True)
	DefaultConnectionGene(key=(-8, 3), weight=-0.10037896524433304, enabled=True)
	DefaultConnectionGene(key=(-7, 0), weight=0.33636660036493343, enabled=False)
	DefaultConnectionGene(key=(-7, 1), weight=0.01576301967088703, enabled=True)
	DefaultConnectionGene(key=(-7, 2), weight=-0.33316717983565663, enabled=True)
	DefaultConnectionGene(key=(-7, 3), weight=-0.07011372695

In [ ]:
env.close()

In [ ]:
import visualize
name = 'winner-{0}'.format(1)
visualize.draw_net(config, genome, view=False, filename=name + "-net.gv")
visualize.draw_net(config, genome, view=False, filename=name + "-net-pruned.gv", prune_unused=True)